# 2 week


In [ ]:
import folium
import geojson
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/final_project/')

%cd "/final_project/MyDrive/final_project"

Drive already mounted at /final_project/; to attempt to forcibly remount, call drive.mount("/final_project/", force_remount=True).
/final_project/MyDrive/final_project


### 1. 
Загрузите агрегированные данные о поездках в мае 2016. Просуммируйте общее количество поездок такси из каждой географической зоны и посчитайте количество ячеек, из которых в мае не было совершено ни одной поездки.

In [ ]:
# загружаем агрегированную таблицу, созданную на прошлой неделе
agg_table = pd.read_csv('agg_table_may_2016.csv', sep=';')

# добавляем к таблице с регионами столбец с количеством вызовов такси
regions = pd.read_csv('regions.csv', sep=';')
regions = regions.set_index('region')
regions['count'] = agg_table.set_index('region').sum(axis=1)

print("количество поездок из каждой географической зоны \n", agg_table.sum(axis=1).sort_values(ascending=False).head())

количество поездок из каждой географической зоны 
 492    640081.0
456    553735.0
419    520804.0
455    490720.0
454    477336.0
dtype: float64


In [ ]:
print("Количество ячеек, из которых в мае не было совершено ни одной поездки", len(regions) - len(agg_table))

Количество ячеек, из которых в мае не было совершено ни одной поездки 1283


### 2. 
Нарисуйте статическую карту Нью-Йорка. Поставьте на карте точку там, где находится Эмпайр-Стейт-Билдинг.


In [ ]:
# сохраняем центр NY
center_ny = [np.mean([40.49612, 40.91553]), np.mean([-74.25559, -73.70001])]

# сохраняем координаты EmpireStateBuilding
ESB_coord = [40.748817, -73.985428]

# рисуем карту. не вижу смысла мучаться с кривыми библиотеками для создания статических карт.
m = folium.Map(location=center_ny, width=500, height=500, zoom_start=10, zoom_control=False, no_touch=True, disable_3d=True)
folium.Marker(ESB_coord, popup="<i>Empire State Building</i>").add_to(m)
m.save('map_static1.html')
m

### 3. 
Поверх статической карты Нью-Йорка визуализируйте данные о поездках из каждой ячейки так, чтобы цветовая шкала, в которую вы окрашиваете каждую ячейку, показывала суммарное количество поездок такси из неё.


In [ ]:
# создаем файл в формате GeoJson
feature_list = []
for i in range(len(regions)):
  # получаем координаты точек для одного квадрата
  geometry = geojson.Polygon([[[regions.iloc[i][0], regions.iloc[i][2]],
                              [regions.iloc[i][0], regions.iloc[i][3]],
                              [regions.iloc[i][1], regions.iloc[i][3]],
                              [regions.iloc[i][1], regions.iloc[i][2]],
                              [regions.iloc[i][0], regions.iloc[i][2]]]])
  
  # получаем номер ячейки
  region_num = regions.iloc[i].name

  # создаем объект Feature
  feature = geojson.Feature(id=str(i), geometry=geometry, properties={'region': str(region_num)})
  feature_list.append(feature)

# создаем объект для координат всех ячеек 
coord = geojson.FeatureCollection(feature_list)

# рисуем карту
m = folium.Map(location=center_ny, width=1000, height=1000, zoom_start=11)
regions['region'] = regions.index.astype(str)
folium.Choropleth(geo_data=coord,
                  data=regions, columns=['region', 'count'],
                  key_on='feature.properties.region',
                  bins=6,
                  fill_color='YlOrRd',
                  nan_fill_color='white', 
                  nan_fill_opacity=0.7,
                  fill_opacity=0.8, 
                  line_opacity=0.1,
                  legend_name='Количество заказов такси').add_to(m)

m.save('mapstatic2.html')
m

### 4. 
Вставьте интерактивную карту Нью-Йорка — такую, которую можно прокручивать и увеличивать. Поставьте метку там, где находится статуя свободы.


Гугл гласит, что координаты статуи свободы - 40.689167, -74.044583

In [ ]:
SoL_coord = [40.689167, -74.044583]            
m = folium.Map(location=center_ny, width=500, height=500, zoom_start=12)
folium.Marker(SoL_coord, popup="<i>Empire State Building</i>").add_to(m)
m

### 5. 
Нарисуйте на интерактивной карте Нью-Йорка ячейки так, чтобы их цвет показывал среднее за месяц количество поездок такси в час из этой зоны.


In [ ]:
# добавляем к таблице с регионами столбец с средним количеством вызовов такси в час
regions['mean_hour_orders'] = (agg_table.set_index('region').sum(axis=1) / len(agg_table.T))

# рисуем карту
m = folium.Map(location=center_ny, width=1000, height=1000, zoom_start=11)
folium.Choropleth(geo_data=coord,
                  data=regions, columns=['region', 'mean_hour_orders'],
                  key_on='feature.properties.region',
                  bins=6,
                  fill_color='YlOrRd',
                  nan_fill_color='white', 
                  nan_fill_opacity=0.7,
                  fill_opacity=0.8, 
                  line_opacity=0.1,
                  legend_name='Количество заказов такси').add_to(m)

m

### 6. 
Чтобы не выбирать из всех 2500 ячеек вручную, отфильтруйте ячейки, из которых в мае совершается в среднем меньше 5 поездок в час. Посчитайте количество оставшихся. Проверьте на карте, что среди этих ячеек нет таких, из которых поездки на самом деле невозможны.


In [ ]:
print('Всего ячеек со средним количеством вызовов больше 5', len(regions[regions['mean_hour_orders'] > 5]))
regions[regions['mean_hour_orders'] > 5].head(5)

Всего ячеек со средним количеством вызовов больше 5 102


,west,east,south,north,count,region,mean_hour_orders
region,,,,,,,
1075,-74.022246,-74.011135,40.697437,40.705825,49699.0,1075,66.710067
1076,-74.022246,-74.011135,40.705825,40.714213,102870.0,1076,138.080537
1077,-74.022246,-74.011135,40.714213,40.722601,64581.0,1077,86.685906
1125,-74.011135,-74.000023,40.697437,40.705825,49135.0,1125,65.953020
1126,-74.011135,-74.000023,40.705825,40.714213,135328.0,1126,181.648322


In [ ]:
regions_more_5 = regions[regions['mean_hour_orders'] > 5]
feature_list = []
for i in range(len(regions_more_5)):
  geometry = geojson.Polygon([[[regions_more_5.iloc[i][0], regions_more_5.iloc[i][2]],
                              [regions_more_5.iloc[i][0], regions_more_5.iloc[i][3]],
                              [regions_more_5.iloc[i][1], regions_more_5.iloc[i][3]],
                              [regions_more_5.iloc[i][1], regions_more_5.iloc[i][2]],
                              [regions_more_5.iloc[i][0], regions_more_5.iloc[i][2]]]])
  region_num = regions_more_5.iloc[i].name
  feature = geojson.Feature(id=str(i), geometry=geometry, properties={'region': str(region_num)})
  feature_list.append(feature)

coord = geojson.FeatureCollection(feature_list)

m = folium.Map(location=center_ny, width=1000, height=1000, zoom_start=11)
folium.Choropleth(geo_data=coord,
                  data=regions_more_5, columns=['region', 'count'],
                  key_on='feature.properties.region',
                  bins=6,
                  fill_color='YlOrRd',
                  nan_fill_color='white', 
                  nan_fill_opacity=0.7,
                  fill_opacity=0.8, 
                  line_opacity=0.1,
                  legend_name='Количество заказов такси').add_to(m)

m

Визуально видно, что все ячейки с количеством вызовов больше 5, находятся в городе, а именно в центральной части Нью-Йорка и в аэропорте